In [42]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, cross_validation, neighbors 
from sklearn import cross_validation, feature_selection, metrics
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression



In [43]:
#importing data
data_set = pd.read_csv("~/Desktop/Dataset_spine.csv",header = 0)
data_set.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att,Unnamed: 13
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN


In [44]:
#data stored per column, x is all the data mines the last column: y, since y contains the classification of the data
#This is code from another solution of this dataset.
data_set['Class_Ord'] = pd.Categorical(data_set.Class_att).codes 

X_Y = data_set[['Col1',
'Col2',
'Col3',
'Col4',
'Col5',
'Col6',
'Col7',
'Col8',
'Col9',
'Col10',
'Col11',
'Col12',
'Class_Ord'
]] 

X = np.array(X_Y.drop(['Class_Ord'], 1))

X_best_features = np.array(X_Y.drop(['Class_Ord', 'Col7',
'Col8',
'Col9',
'Col10',
'Col11',
'Col12',
'Col4',
],
1))

#The outlier in row 116 is removed
X_no_outlier = np.delete(X_best_features,(115),axis=0)

Y = np.array(data_set['Class_Ord'])

Y_no_outlier = np.delete(Y,(115),axis=0)


In [ ]:
#running this will give a graph in an external window, this needs to be closed again before anything else can be run!!
######################################################################################################################


#empty list that will contain the predicted accuracies
predict_accuracy_list = []
#starting values necessary for the while loop
k = 1
n = 1
#changable variables, number of iterations and number of nearest-neighbours 
amount_of_iterations = 1000
amount_of_k = 50

#runs n times the while loop that predicts accuracies for different k's.
#returns a n*k list of predicted accuracies.
while n<=amount_of_iterations:
    #creating test and training datasets, X_beste_features, to use the dataset without the useless features.
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_no_outlier, Y_no_outlier, test_size = 0.2)
    while k <= amount_of_k:
        #preparing the classifier
        clf = neighbors.KNeighborsClassifier(n_neighbors = k, weights='distance',)
        clf.fit(X_train, y_train)
        #calculating the predict accuracy
        predict_accuracy = clf.score(X_test, y_test)
        k = k+1
        predict_accuracy_list = np.append(predict_accuracy_list, predict_accuracy) 
    k = 1
    n = n+1

    
#row vector of increasing integers of length k
amount_of_k_matrix = np.matrix(np.arange(1, amount_of_k+1, dtype=np.float))

#row vector of length n*k, containing all predicted accuracies
prediction_matrix = np.matrix(predict_accuracy_list)

#prediction_matrix is reshaped to shape n x k
reshaped_prediction_matrix = prediction_matrix.reshape(amount_of_iterations,amount_of_k)

#plots a graph with the number of k used on the x-axis and respectively the predicted accuracy
#over n iterations on the y-axis
plt.plot(amount_of_k_matrix, reshaped_prediction_matrix.mean(0), 'x')
plt.show()


In [33]:
#running this will give a graph in an external window, this needs to be closed again before anything else can be run!!
######################################################################################################################

#Finding most important features 
kb = feature_selection.SelectKBest()

#returns two arrays; score and p_value
score_pvalue = np.matrix(kb.score_func(X,Y))

#plotting the score on the y-axis and the features on the x-axis
score = score_pvalue.T[:,0]
columns = np.arange(1,13,dtype = np.float)
plt.plot(columns, score, 'x')
plt.show()

In [41]:
#Finding f1_score; 2*(precision*recall)/(precision+recall)

#ran with optimum k=9, for dataset with feature selection
i= 1
f1_l = []
predict_class1 = []
while i<=1000:
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_best_features, Y, test_size = 0.2)
    clf1 = neighbors.KNeighborsClassifier(n_neighbors = 9)
    clf1.fit(X_train, y_train)
    predict_class1 = clf1.predict(X_test)
    f1 = metrics.f1_score(y_test, predict_class1)
    f1_l = np.append(f1_l, f1)
    i = i+1
    
    
print "f1-score best_features:", np.average(f1_l)


#ran with optimum k=27, for datatest without feature selection
j = 1
f2_l = []
predict_class2 = []
while j<=1000:
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size = 0.2)
    clf2 = neighbors.KNeighborsClassifier(n_neighbors = 27)
    clf2.fit(X_train, y_train)
    predict_class = clf2.predict(X_test)
    f2 = metrics.f1_score(y_test, predict_class)
    f2_l = np.append(f2_l, f1)
    j = j+1
    
print "f1-score all features:", np.average(f2_l)
#0.766933945719




cf_matrix = metrics.confusion_matrix(y_test, predict_class1)
print "confusion matrix:"
print cf_matrix



f1-score best_features: 0.766152007147
f1-score all features: 0.611111111111
[]
confusion matrix:
[[32 12]
 [14  4]]
